In [5]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve#,scorer
import dill
dill._dill._reverse_typemap['ClassType'] = type

In [6]:
df = pd.read_csv('./data/Corona_NLP_test.csv', encoding='ISO-8859-1')


In [7]:
y_test = df["Sentiment"]

In [8]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [9]:
import re
import numpy as np
# write function for removing @user
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt
df['Tweet'] = np.vectorize(remove_pattern)(df['OriginalTweet'], '@[\w]*')
df['Tweet'] = df['Tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df['Tweet'] = df['Tweet'].str.replace('[^a-zA-Z#]+',' ')
df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))
tokenized_tweet = df['Tweet'].apply(lambda x: x.split())

from nltk.stem.porter import *
stemmer = PorterStemmer()

# apply stemmer for tokenized_tweet
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])
new_df = df[['Tweet','Sentiment']]
new_df.head()
new_df["Tweet"] = new_df["Tweet"].str.lower()

C:\Users\Home\AppData\Local\Temp/ipykernel_28580/1612159636.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet'] = df['Tweet'].str.replace('[^a-zA-Z#]+',' ')
C:\Users\Home\AppData\Local\Temp/ipykernel_28580/1612159636.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Tweet"] = new_df["Tweet"].str.lower()


In [10]:
new_df.head()

,Tweet,Sentiment
0,trending new yorkers encounter empty supermark...,Extremely Negative
1,when couldn find hand sanitizer fred meyer tur...,Positive
2,find out how you can protect yourself and love...,Extremely Positive
3,#panic buying hits #newyork city anxious shopp...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [11]:
X_test = new_df

In [12]:
X_test.head()

,Tweet,Sentiment
0,trending new yorkers encounter empty supermark...,Extremely Negative
1,when couldn find hand sanitizer fred meyer tur...,Positive
2,find out how you can protect yourself and love...,Extremely Positive
3,#panic buying hits #newyork city anxious shopp...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [13]:
with open('LogReg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [14]:
predictions = pipeline.predict(X_test)
pd.DataFrame({'preds': predictions}).to_csv("test_predictions.csv", index=None)

C:\Users\Home\AppData\Local\Temp/ipykernel_26004/249066176.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report( predictions, y_test))
#print(confusion_matrix(y_test, predictions))


                    precision    recall  f1-score   support

Extremely Negative       0.11      0.69      0.19        97
Extremely Positive       0.17      0.85      0.29       123
          Negative       0.49      0.41      0.45      1263
           Neutral       0.23      0.57      0.32       246
          Positive       0.76      0.35      0.48      2069

          accuracy                           0.41      3798
         macro avg       0.35      0.57      0.35      3798
      weighted avg       0.60      0.41      0.45      3798

